# Data and problem discription

In [24]:
# Load and inspect disaster demand data
import pandas as pd
from pathlib import Path
from IPython.display import display

DATA_PATH = Path("hospital_disaster_dataset.csv")
df = pd.read_csv(DATA_PATH)

hospital_info = (
    df[
        [
            "hospital_id",
            "hospital_name",
            "capacity_beds",
            "allocation_cost_per_unit",
        ]
    ]
    .drop_duplicates()
    .set_index("hospital_id")
)
scenario_info = (
    df[["scenario_id", "disaster_type", "scenario_probability"]]
    .drop_duplicates()
    .set_index("scenario_id")
    .sort_index()
)
demand = (
    df.pivot_table(
        index="scenario_id", columns="hospital_id", values="demand", aggfunc="first"
    )
    .reindex(scenario_info.index)
    .astype(float)
)

print(f"Loaded {len(df)} rows with {len(hospital_info)} hospitals and {len(scenario_info)} scenarios.")
display(df.head())

expected_demand = demand.mul(scenario_info["scenario_probability"], axis=0).sum()
summary_table = pd.concat(
    [
        hospital_info[["hospital_name", "capacity_beds"]],
        expected_demand.rename("expected_demand"),
    ],
    axis=1,
)
summary_table["capacity_utilization_at_expectation"] = (
    summary_table["expected_demand"] / summary_table["capacity_beds"]
)
summary_table = summary_table.round({"expected_demand": 2, "capacity_utilization_at_expectation": 3})

print("\nExpected demand by hospital (weighted by scenario probabilities):")
display(summary_table)

print("Scenario probabilities sum to:", scenario_info["scenario_probability"].sum())

Loaded 24 rows with 3 hospitals and 8 scenarios.


,scenario_id,scenario_probability,disaster_type,hospital_id,hospital_name,capacity_beds,demand,allocation_cost_per_unit,shortage_penalty_per_unit
0,S0,0.65,no_disaster,H1,Central Hospital,250,6,2.0,8.0
1,S0,0.65,no_disaster,H2,North Clinic,180,5,2.2,8.0
2,S0,0.65,no_disaster,H3,South Medical Center,150,4,1.8,8.0
3,S1,0.05,mild_flood,H1,Central Hospital,250,60,2.0,8.0
4,S1,0.05,mild_flood,H2,North Clinic,180,45,2.2,8.0



Expected demand by hospital (weighted by scenario probabilities):


,hospital_name,capacity_beds,expected_demand,capacity_utilization_at_expectation
hospital_id,,,,
H1,Central Hospital,250,37.55,0.150
H2,North Clinic,180,28.80,0.160
H3,South Medical Center,150,22.65,0.151


Scenario probabilities sum to: 1.0


## Problem description
We manage a network of hospitals that can pre-position a limited pool of emergency medical resource units (beds, staff teams, critical kits). Nature reveals one of seven discrete disaster scenarios, each with a probability from the historical record. Every hospital can host only up to its surge capacity, and falling short of demand incurs a penalty because patients must be diverted or left untreated. Allocation decisions must balance transportation/activation cost with the risk of shortages under high-impact scenarios.

### Modeling assumptions
- Total pre-positionable stock is capped at 220 units system-wide (mobile response teams, beds, ventilators, etc.).
- Additional "surge" deployments can be flown in post-disaster, limited to 120 units per scenario and costing 35% more than pre-positioning because of overtime logistics.
- Allocation cost coefficients in the dataset act as a per-unit handling/operating cost that varies by hospital.
- Shortage penalty (8 cost units) captures reputational, regulatory, and humanitarian impacts of unmet demand.

# Models

In [25]:
# Shared parameters and helpers for all models
try:
    import gurobipy as gp
    from gurobipy import GRB
except ImportError as exc:
    raise ImportError(
        "The Gurobi optimizer is required for these models.\n"
        "Install gurobipy in the active environment and ensure you have a valid license."
    ) from exc

TOTAL_PREPOSITION_STOCK = 220
SURGE_STOCK = 120
SURGE_COST_MULTIPLIER = 1.35
SHORTAGE_PENALTY = float(df["shortage_penalty_per_unit"].iloc[0])

allocation_cost = hospital_info["allocation_cost_per_unit"].to_dict()
capacity = hospital_info["capacity_beds"].to_dict()
scenario_prob = scenario_info["scenario_probability"].to_dict()

# demand.loc[scenario, hospital] already holds deterministic requirements


def _solution_to_frame(variable_dict, index_labels, value_name):
    """Convert a {(keys): var} dictionary into a tidy DataFrame."""
    records = []
    for key, var in variable_dict.items():
        value = var.X
        if abs(value) <= 1e-6:
            continue
        key = key if isinstance(key, tuple) else (key,)
        records.append(dict(zip(index_labels, key), **{value_name: value}))
    return pd.DataFrame(records).set_index(index_labels) if records else pd.DataFrame()


def solve_and_report(model, description):
    model.Params.OutputFlag = 0
    model.optimize()
    if model.Status != GRB.OPTIMAL:
        raise RuntimeError(f"{description} did not converge: status {model.Status}")
    print(f"{description} optimal objective: {model.ObjVal:,.2f}")
    return model

In [26]:
# Perfect-foresight upper bound LP (scenario-specific decisions)
scenarios = list(scenario_info.index)
hospitals = list(hospital_info.index)

lp_model = gp.Model("LP_expectation")
alloc = lp_model.addVars(scenarios, hospitals, name="alloc", lb=0)
shortage = lp_model.addVars(scenarios, hospitals, name="shortage", lb=0)

for s in scenarios:
    lp_model.addConstr(
        gp.quicksum(alloc[s, h] for h in hospitals) <= TOTAL_PREPOSITION_STOCK,
        name=f"total_stock_{s}",
    )
    for h in hospitals:
        lp_model.addConstr(alloc[s, h] <= capacity[h], name=f"cap_{s}_{h}")
        lp_model.addConstr(
            alloc[s, h] + shortage[s, h] == demand.loc[s, h], name=f"balance_{s}_{h}"
        )

lp_model.setObjective(
    gp.quicksum(
        scenario_prob[s]
        * (
            gp.quicksum(
                allocation_cost[h] * alloc[s, h]
                + SHORTAGE_PENALTY * shortage[s, h]
                for h in hospitals
            )
        )
        for s in scenarios
    ),
    GRB.MINIMIZE,
)

lp_model = solve_and_report(lp_model, "Perfect-foresight upper bound")

lp_alloc_df = _solution_to_frame(alloc, ["scenario_id", "hospital_id"], "resources")
lp_shortage_df = _solution_to_frame(shortage, ["scenario_id", "hospital_id"], "shortage")

print("\nScenario-dependent allocation plan (units > 0 only):")
display(lp_alloc_df)

print("\nExpected shortage contribution by scenario/hospital:")
display(lp_shortage_df)

lp_expected_shortage = sum(
    scenario_prob[s] * sum(shortage[s, h].X for h in hospitals) for s in scenarios
)
print(f"Weighted average shortage (units): {lp_expected_shortage:.2f}")

Expectation LP optimal objective: 250.84

Scenario-dependent allocation plan (units > 0 only):


resources
scenario_id hospital_id           
S0          H1                 6.0
            H2                 5.0
            H3                 4.0
S1          H1                60.0
            H2                45.0
            H3                35.0
S2          H1                85.0
            H2                65.0
            H3                45.0
S3          H1               110.0
            H2                45.0
            H3                65.0
S4          H1                70.0
            H2                55.0
            H3                45.0
S5          H1                95.0
            H2                70.0
            H3                55.0
S6          H1               130.0
            H3                90.0
S7          H1               100.0
            H3               120.0


Expected shortage contribution by scenario/hospital:


shortage
scenario_id hospital_id          
S3          H2               40.0
S5          H2                5.0
S6          H1               20.0
            H2              110.0
S7          H1              100.0
            H2              140.0

Weighted average shortage (units): 12.25


In [27]:
# SP model (two-stage stochastic programming)
sp_model = gp.Model("two_stage_stochastic")
preposition = sp_model.addVars(hospitals, name="pre", lb=0)
surge = sp_model.addVars(scenarios, hospitals, name="surge", lb=0)
shortage_sp = sp_model.addVars(scenarios, hospitals, name="shortage", lb=0)

sp_model.addConstr(gp.quicksum(preposition[h] for h in hospitals) <= TOTAL_PREPOSITION_STOCK, name="stock_limit")
for h in hospitals:
    sp_model.addConstr(preposition[h] <= capacity[h], name=f"pre_cap_{h}")

for s in scenarios:
    sp_model.addConstr(
        gp.quicksum(surge[s, h] for h in hospitals) <= SURGE_STOCK,
        name=f"surge_pool_{s}",
    )
    for h in hospitals:
        sp_model.addConstr(preposition[h] + surge[s, h] <= capacity[h], name=f"cap_{s}_{h}")
        sp_model.addConstr(
            preposition[h] + surge[s, h] + shortage_sp[s, h] == demand.loc[s, h],
            name=f"balance_{s}_{h}",
        )

first_stage_cost = gp.quicksum(allocation_cost[h] * preposition[h] for h in hospitals)
recourse_cost = gp.quicksum(
    scenario_prob[s]
    * gp.quicksum(
        SURGE_COST_MULTIPLIER * allocation_cost[h] * surge[s, h]
        + SHORTAGE_PENALTY * shortage_sp[s, h]
        for h in hospitals
    )
    for s in scenarios
)

sp_model.setObjective(first_stage_cost + recourse_cost, GRB.MINIMIZE)
sp_model = solve_and_report(sp_model, "Two-stage SP")

preposition_df = _solution_to_frame(preposition, ["hospital_id"], "preposition")
surge_df = _solution_to_frame(surge, ["scenario_id", "hospital_id"], "surge")
shortage_sp_df = _solution_to_frame(shortage_sp, ["scenario_id", "hospital_id"], "shortage")

print("\nOptimal pre-positioning plan:")
display(preposition_df)

print("\nScenario-specific surge deployments:")
display(surge_df)

print("\nScenario-specific shortages:")
display(shortage_sp_df)

sp_expected_shortage = sum(
    scenario_prob[s] * sum(shortage_sp[s, h].X for h in hospitals) for s in scenarios
)
print(f"Weighted average shortage (units): {sp_expected_shortage:.2f}")

Two-stage SP optimal objective: 395.44

Optimal pre-positioning plan:


,preposition
hospital_id,
H1,6.0
H2,5.0
H3,4.0



Scenario-specific surge deployments:


surge
scenario_id hospital_id       
S1          H1            54.0
            H2            35.0
            H3            31.0
S2          H1            79.0
            H3            41.0
S3          H1            59.0
            H3            61.0
S4          H1            64.0
            H2            15.0
            H3            41.0
S5          H1            69.0
            H3            51.0
S6          H1            34.0
            H3            86.0
S7          H1             4.0
            H3           116.0


Scenario-specific shortages:


shortage
scenario_id hospital_id          
S1          H2                5.0
S2          H2               60.0
S3          H1               45.0
            H2               80.0
S4          H2               35.0
S5          H1               20.0
            H2               70.0
S6          H1              110.0
            H2              105.0
S7          H1              190.0
            H2              135.0

Weighted average shortage (units): 32.00


In [28]:
# OR model (deterministic optimization on expected demand)
or_model = gp.Model("deterministic_expected")
deterministic_alloc = or_model.addVars(hospitals, name="alloc", lb=0)
deterministic_shortage = or_model.addVars(hospitals, name="shortage", lb=0)

or_model.addConstr(
    gp.quicksum(deterministic_alloc[h] for h in hospitals) <= TOTAL_PREPOSITION_STOCK,
    name="stock_limit",
)
for h in hospitals:
    or_model.addConstr(deterministic_alloc[h] <= capacity[h], name=f"cap_{h}")
    or_model.addConstr(
        deterministic_alloc[h] + deterministic_shortage[h] == expected_demand[h],
        name=f"balance_{h}",
    )

or_model.setObjective(
    gp.quicksum(
        allocation_cost[h] * deterministic_alloc[h]
        + SHORTAGE_PENALTY * deterministic_shortage[h]
        for h in hospitals
    ),
    GRB.MINIMIZE,
)

or_model = solve_and_report(or_model, "Deterministic OR")

deterministic_alloc_df = _solution_to_frame(
    deterministic_alloc, ["hospital_id"], "allocation"
)
deterministic_shortage_df = _solution_to_frame(
    deterministic_shortage, ["hospital_id"], "shortage"
)

print("\nDeterministic allocation plan based on expected demand:")
display(deterministic_alloc_df)

print("\nResidual shortage under expected demand:")
display(deterministic_shortage_df)

print(
    "Total deterministic shortage:",
    sum(deterministic_shortage[h].X for h in hospitals),
)

Deterministic OR optimal objective: 179.23

Deterministic allocation plan based on expected demand:


,allocation
hospital_id,
H1,37.55
H2,28.80
H3,22.65



Residual shortage under expected demand:


""


Total deterministic shortage: 0.0


In [29]:
# RO model (robust optimization with budgeted worst-case shortage)
ro_model = gp.Model("robust_allocation")
robust_alloc = ro_model.addVars(hospitals, name="alloc", lb=0)
robust_shortage = ro_model.addVars(scenarios, hospitals, name="shortage", lb=0)
worst_shortage = ro_model.addVar(name="worst_case_shortage", lb=0)

ro_model.addConstr(
    gp.quicksum(robust_alloc[h] for h in hospitals) <= TOTAL_PREPOSITION_STOCK,
    name="stock_limit",
)
for h in hospitals:
    ro_model.addConstr(robust_alloc[h] <= capacity[h], name=f"cap_{h}")

for s in scenarios:
    scenario_shortage_expr = gp.quicksum(robust_shortage[s, h] for h in hospitals)
    ro_model.addConstr(scenario_shortage_expr <= worst_shortage, name=f"worst_link_{s}")
    for h in hospitals:
        ro_model.addConstr(
            robust_alloc[h] + robust_shortage[s, h] >= demand.loc[s, h],
            name=f"coverage_{s}_{h}",
        )

allocation_cost_expr = gp.quicksum(allocation_cost[h] * robust_alloc[h] for h in hospitals)
ro_model.setObjective(allocation_cost_expr + SHORTAGE_PENALTY * worst_shortage, GRB.MINIMIZE)

ro_model = solve_and_report(ro_model, "Robust OR")

robust_alloc_df = _solution_to_frame(robust_alloc, ["hospital_id"], "allocation")
robust_shortage_df = _solution_to_frame(
    robust_shortage, ["scenario_id", "hospital_id"], "shortage"
)

print("\nRobust allocation plan (single plan for all scenarios):")
display(robust_alloc_df)

print("\nScenario shortage incurred under the robust plan:")
display(robust_shortage_df)

print(
    f"Worst-case shortage across all scenarios: {worst_shortage.X:.2f} units"
)

Robust OR optimal objective: 2,336.00

Robust allocation plan (single plan for all scenarios):


,allocation
hospital_id,
H1,100.0
H3,120.0



Scenario shortage incurred under the robust plan:


shortage
scenario_id hospital_id          
S0          H1              235.0
            H2                5.0
S1          H1              195.0
            H2               45.0
S2          H1              175.0
            H2               65.0
S3          H1              155.0
            H2               85.0
S4          H1              185.0
            H2               55.0
S5          H1              165.0
            H2               75.0
S6          H1              130.0
            H2              110.0
S7          H1              100.0
            H2              140.0

Worst-case shortage across all scenarios: 240.00 units


# Evaluate

In [30]:
# Compare model performance metrics
import numpy as np


def _scenario_shortage_dict(shortage_vars):
    return {
        s: sum(shortage_vars[s, h].X for h in hospitals)
        for s in scenarios
    }


lp_shortage_by_scenario = _scenario_shortage_dict(shortage)
sp_shortage_by_scenario = _scenario_shortage_dict(shortage_sp)
ro_shortage_by_scenario = _scenario_shortage_dict(robust_shortage)

# Deterministic plan evaluated against all scenarios (no recourse allowed here)
deterministic_plan = {h: deterministic_alloc[h].X for h in hospitals}
det_shortage_by_scenario = {}
for s in scenarios:
    short = 0
    for h in hospitals:
        served = min(deterministic_plan[h], capacity[h])
        short += max(0.0, demand.loc[s, h] - served)
    det_shortage_by_scenario[s] = short


def weighted_metrics(shortage_dict):
    weighted = sum(scenario_prob[s] * shortage_dict[s] for s in scenarios)
    worst_case = max(shortage_dict.values())
    perc95 = np.quantile(list(shortage_dict.values()), 0.95)
    return weighted, worst_case, perc95


lp_weighted, lp_worst, lp_p95 = weighted_metrics(lp_shortage_by_scenario)
sp_weighted, sp_worst, sp_p95 = weighted_metrics(sp_shortage_by_scenario)
ro_weighted, ro_worst, ro_p95 = weighted_metrics(ro_shortage_by_scenario)

det_weighted, det_worst, det_p95 = weighted_metrics(det_shortage_by_scenario)

overall_eval = pd.DataFrame(
    [
        {
            "model": "Perfect-foresight UB",
            "objective": lp_model.ObjVal,
            "expected_shortage": lp_weighted,
            "worst_case_shortage": lp_worst,
            "p95_shortage": lp_p95,
        },
        {
            "model": "Two-stage SP",
            "objective": sp_model.ObjVal,
            "expected_shortage": sp_weighted,
            "worst_case_shortage": sp_worst,
            "p95_shortage": sp_p95,
        },
        {
            "model": "Robust OR",
            "objective": ro_model.ObjVal,
            "expected_shortage": ro_weighted,
            "worst_case_shortage": ro_worst,
            "p95_shortage": ro_p95,
        },
        {
            "model": "Deterministic OR",
            "objective": or_model.ObjVal,
            "expected_shortage": det_weighted,
            "worst_case_shortage": det_worst,
            "p95_shortage": det_p95,
        },
    ]
).set_index("model")

print("Model comparison (lower is better):")
display(overall_eval.round(2))

print("\nScenario-level shortage (units) for reference:")
comparison = pd.DataFrame(
    {
        "scenario_probability": scenario_info["scenario_probability"],
        "UpperBound_shortage": pd.Series(lp_shortage_by_scenario),
        "SP_shortage": pd.Series(sp_shortage_by_scenario),
        "RO_shortage": pd.Series(ro_shortage_by_scenario),
        "Deterministic_shortage": pd.Series(det_shortage_by_scenario),
    }
)
display(comparison)

print(
    f"Worst-case shortage guaranteed by robust plan: {worst_shortage.X:.2f} units"
)

Model comparison (lower is better):


,objective,expected_shortage,worst_case_shortage,p95_shortage
model,,,,
Expectation LP,250.84,12.25,240.0,201.5
Two-stage SP,395.44,32.00,325.0,286.5
Robust OR,2336.00,240.00,240.0,240.0
Deterministic OR,179.23,48.10,371.0,332.5



Scenario-level shortage (units) for reference:


,scenario_probability,LP_shortage,SP_shortage,RO_shortage,Deterministic_shortage
S0,0.65,0.0,0.0,240.0,0.0
S1,0.05,0.0,5.0,240.0,51.0
S2,0.04,0.0,60.0,240.0,106.0
S3,0.05,40.0,125.0,240.0,171.0
S4,0.10,0.0,35.0,240.0,81.0
S5,0.05,5.0,90.0,240.0,136.0
S6,0.04,130.0,215.0,240.0,261.0
S7,0.02,240.0,325.0,240.0,371.0


Worst-case shortage guaranteed by robust plan: 240.00 units
